### Kaynak
https://www.veribilimiokulu.com/support-vector-machine-svm-ile-siniflandirma-python-ornek-uygulamasi/  
https://bilgisayarkavramlari.com/2013/03/31/siniflandirma-classification/

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
import tensorflow as tf
from scipy.io import wavfile as wav
import warnings
warnings.filterwarnings('ignore')

In [2]:
audio_dataset_path='sound_split/sounds/'
metadata=pd.read_csv('sound_split/Splitmetadata.csv')
mfcc = 70 # öznitelik sayısı
num_labels = 7 # Sınıf sayısı

In [3]:
def features_extractor(filename):
    
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=mfcc)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [4]:
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["file_name"]))
    final_class_labels=row["class_name"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

5178it [07:22, 11.70it/s]


In [5]:
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])

X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [44]:
cls = extracted_features_df['class'].tolist()
X_mean = []
X_min = []
X_max = []
for i in X:
    X_mean.append(np.mean(i))
    X_min.append(np.min(i))
    X_max.append(np.max(i))

In [49]:
with open("SVM_lehce.txt","a") as file:
    file.write("mean,min,max,class\n")
    for i in range(0,len(X_mean)):
        file.write(str(X_mean[i])+","+str(X_min[i])+","+str(X_max[i])+","+cls[i]+"\n")

In [50]:
dataset = pd.read_csv('SVM_lehce.csv')

In [51]:
dataset

,mean,min,max,class
0,-4.032666,-302.22950,115.561800,turkmence
1,-4.325071,-302.14243,115.484375,turkmence
2,-4.159544,-272.86038,117.178750,turkmence
3,-4.167392,-282.07697,133.300640,turkmence
4,-4.028280,-269.52304,128.307890,turkmence
...,...,...,...,...
5173,-4.168431,-383.62125,116.744736,ozbekce
5174,-4.105752,-394.87085,117.603560,ozbekce
5175,-4.240935,-427.92280,113.422180,ozbekce
5176,-8.412517,-735.09110,59.621376,ozbekce


In [52]:
X = dataset.iloc[:, [0,1,2]].values
y = dataset.iloc[:, 3].values

In [53]:
X

array([[   -4.0326657,  -302.2295   ,   115.5618   ],
       [   -4.325071 ,  -302.14243  ,   115.484375 ],
       [   -4.159544 ,  -272.86038  ,   117.17875  ],
       ...,
       [   -4.240935 ,  -427.9228   ,   113.42218  ],
       [   -8.412517 ,  -735.0911   ,    59.621376 ],
       [  -16.162441 , -1131.3708   ,     0.       ]])

In [54]:
y

array(['turkmence', 'turkmence', 'turkmence', ..., 'ozbekce', 'ozbekce',
       'ozbekce'], dtype=object)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [56]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [57]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [58]:
y_pred = classifier.predict(X_test)

In [59]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 62  28   4   2   0  32  76]
 [  8 150   0   0   0  29  10]
 [  4  14  11   0   0  40  76]
 [  8   6   5  42   0  40  45]
 [ 30   2   3   1   0  40  20]
 [  1  13   1   1   0 208  29]
 [  4  19   0   5   0   4 222]]


In [60]:
y_pred

array(['kirgizca', 'turkmence', 'uygurca', ..., 'uygurca', 'turkmence',
       'uygurca'], dtype=object)

In [61]:
y_test

array(['tatarca', 'turkmence', 'azerbaycan', ..., 'ozbekce', 'kirgizca',
       'azerbaycan'], dtype=object)